# Lecture Notes Generation Hackathon

This notebook implements a comprehensive solution for automatically generating lecture notes from multimedia content. The system processes video lectures and PowerPoint presentations to produce well-structured, informative notes using AI-based transcription and language models.

## 1. Import Required Libraries

In [16]:
# Core Python libraries
import os
import sys
import json
import time
import glob
from pathlib import Path

# Multimedia processing
import ffmpeg
import cv2
import librosa
import soundfile as sf
from pydub import AudioSegment

# PowerPoint and document processing
from pptx import Presentation
import docx
import fitz  # PyMuPDF for PDF processing

# Data handling and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# AI and ML libraries
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    AutoModelForSpeechSeq2Seq, 
    AutoProcessor,
    pipeline
)
from datasets import Audio
from huggingface_hub import login

# API clients
import openai
import groq

# Web interface (optional)
import gradio as gr
import streamlit as st

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Dataset Preparation

Here we'll organize the video lectures and presentation files into a structured directory for efficient batch processing. 

In [17]:
# Define base directories
BASE_DIR = Path("./data")
RAW_DIR = BASE_DIR / "raw"
PROCESSED_DIR = BASE_DIR / "processed"
OUTPUT_DIR = BASE_DIR / "output"

# Create directory structure if it doesn't exist
for directory in [BASE_DIR, RAW_DIR, PROCESSED_DIR, OUTPUT_DIR]:
    directory.mkdir(exist_ok=True, parents=True)

# Define subdirectories for specific content types
VIDEO_DIR = RAW_DIR / "videos"
SLIDES_DIR = RAW_DIR / "slides"
AUDIO_DIR = PROCESSED_DIR / "audio"
TRANSCRIPTS_DIR = PROCESSED_DIR / "transcripts"
NOTES_DIR = OUTPUT_DIR / "lecture_notes"

for directory in [VIDEO_DIR, SLIDES_DIR, AUDIO_DIR, TRANSCRIPTS_DIR, NOTES_DIR]:
    directory.mkdir(exist_ok=True, parents=True)

print("Directory structure created!")

# Function to scan and index available content
def scan_content():
    video_files = list(VIDEO_DIR.glob("*.mp4")) + list(VIDEO_DIR.glob("*.avi"))
    slide_files = list(SLIDES_DIR.glob("*.pptx")) + list(SLIDES_DIR.glob("*.ppt"))
    
    content_index = {
        "videos": [str(file.name) for file in video_files],
        "slides": [str(file.name) for file in slide_files]
    }
    
    print(f"Found {len(content_index['videos'])} video files and {len(content_index['slides'])} slide files")
    return content_index, video_files, slide_files

# Scan for content (will be used after data is added)
content_index, video_files, slide_files = scan_content()

Directory structure created!
Found 7 video files and 8 slide files


In [7]:
# Function to organize and copy the LLM_DATASET into our structured directories
def organize_dataset(dataset_path="LLM_DATASET"):
    """Organize and copy files from the LLM_DATASET into the structured data directory"""
    import shutil
    from pathlib import Path
    
    dataset_path = Path(dataset_path)
    if not dataset_path.exists():
        print(f"Dataset path not found: {dataset_path}")
        return False
    
    print(f"Organizing dataset from: {dataset_path}")
    
    # Track copied files
    copied_videos = []
    copied_slides = []
    
    # Process all subdirectories in the dataset
    for topic_dir in dataset_path.iterdir():
        if not topic_dir.is_dir():
            continue
            
        topic_name = topic_dir.name
        print(f"Processing topic: {topic_name}")
        
        # Copy video files
        video_files = list(topic_dir.glob("*.mp4")) + list(topic_dir.glob("*.mkv")) + list(topic_dir.glob("*.avi"))
        for video_file in video_files:
            # Create a destination filename with topic prefix for organization
            dest_filename = f"{topic_name}_{video_file.name}"
            dest_path = VIDEO_DIR / dest_filename
            
            # Copy the file
            shutil.copy2(video_file, dest_path)
            copied_videos.append(dest_filename)
            print(f"  Copied video: {video_file.name} → {dest_filename}")
        
        # Copy slide files (PowerPoint and PDF)
        slide_files = list(topic_dir.glob("*.pptx")) + list(topic_dir.glob("*.ppt")) + list(topic_dir.glob("*.pdf"))
        for slide_file in slide_files:
            # Create a destination filename with topic prefix
            dest_filename = f"{topic_name}_{slide_file.name}"
            dest_path = SLIDES_DIR / dest_filename
            
            # Copy the file
            shutil.copy2(slide_file, dest_path)
            copied_slides.append(dest_filename)
            print(f"  Copied slides: {slide_file.name} → {dest_filename}")
    
    print(f"\nDataset organization complete!")
    print(f"Copied {len(copied_videos)} video files and {len(copied_slides)} slide files")
    
    # Refresh content index after copying
    return scan_content()

# Uncomment and run the below line to organize your dataset
content_index, video_files, slide_files = organize_dataset("LLM_DATASET")

Organizing dataset from: LLM_DATASET
Processing topic: Lora&Qlora
  Copied video: 19853_shylaja.sharath_31_20250318121200085_Video_ENC (1).mp4 → Lora&Qlora_19853_shylaja.sharath_31_20250318121200085_Video_ENC (1).mp4
  Copied video: 19853_shylaja.sharath_31_20250318112700094_Video_ENC (1).mp4 → Lora&Qlora_19853_shylaja.sharath_31_20250318112700094_Video_ENC (1).mp4
  Copied video: 19853_shylaja.sharath_31_20250318112700094_Video_ENC (1).mp4 → Lora&Qlora_19853_shylaja.sharath_31_20250318112700094_Video_ENC (1).mp4
  Copied video: 19853_shylaja.sharath_31_20250318125700082_Video_ENC.mp4 → Lora&Qlora_19853_shylaja.sharath_31_20250318125700082_Video_ENC.mp4
  Copied slides: Finetuning.pdf → Lora&Qlora_Finetuning.pdf
Processing topic: Tree_traversal
  Copied video: 19853_shylaja.sharath_31_20250318125700082_Video_ENC.mp4 → Lora&Qlora_19853_shylaja.sharath_31_20250318125700082_Video_ENC.mp4
  Copied slides: Finetuning.pdf → Lora&Qlora_Finetuning.pdf
Processing topic: Tree_traversal
  Copied 

In [8]:
# Function to identify matching slide and video pairs
def find_content_pairs():
    """Find matching pairs of videos and slides based on topic/filename"""
    # Refresh content indexes first
    content_index, video_files, slide_files = scan_content()
    
    # Convert to string paths for easier matching
    video_paths = [str(v) for v in video_files]
    slide_paths = [str(s) for s in slide_files]
    
    pairs = []
    unpaired_videos = []
    unpaired_slides = []
    
    # Find pairs by matching topic prefixes
    for video_path in video_paths:
        video_name = os.path.basename(video_path)
        topic_prefix = video_name.split('_')[0]  # Get the topic prefix
        
        # Find slides with matching topic
        matching_slides = [s for s in slide_paths if os.path.basename(s).startswith(topic_prefix)]
        
        if matching_slides:
            # For simplicity, take the first match (can be improved with better matching logic)
            pairs.append({
                "topic": topic_prefix,
                "video": video_path,
                "slides": matching_slides[0]
            })
        else:
            unpaired_videos.append(video_path)
    
    # Find slides without video pairs
    paired_slide_paths = [p["slides"] for p in pairs]
    unpaired_slides = [s for s in slide_paths if s not in paired_slide_paths]
    
    print(f"Found {len(pairs)} content pairs with matching videos and slides")
    print(f"Found {len(unpaired_videos)} videos without matching slides")
    print(f"Found {len(unpaired_slides)} slides without matching videos")
    
    return pairs, unpaired_videos, unpaired_slides

# Uncomment to find content pairs
content_pairs, unpaired_videos, unpaired_slides = find_content_pairs()

Found 7 video files and 8 slide files
Found 0 content pairs with matching videos and slides
Found 7 videos without matching slides
Found 8 slides without matching videos


## 3. Framework Design and Environment Setup

Set up the development environment with necessary configurations for video decoding, audio processing, and document parsing.

In [9]:
# Check system capabilities
def check_environment():
    system_info = {
        "python_version": sys.version,
        "cuda_available": torch.cuda.is_available(),
        "cuda_device_count": torch.cuda.device_count() if torch.cuda.is_available() else 0,
        "ffmpeg_available": os.system("ffmpeg -version > /dev/null 2>&1") == 0
    }
    
    if system_info["cuda_available"]:
        system_info["cuda_device_names"] = [torch.cuda.get_device_name(i) for i in range(system_info["cuda_device_count"])]
    
    print("System configuration:")
    for key, value in system_info.items():
        print(f"- {key}: {value}")
    
    return system_info

system_info = check_environment()

# Set the device for PyTorch operations
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

System configuration:
- python_version: 3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:54:21) [Clang 16.0.6 ]
- cuda_available: False
- cuda_device_count: 0
- ffmpeg_available: True
Using device: cpu


## 3.1 Batch Audio Extraction

Now that we have our dataset organized, let's extract audio from all videos at once.

In [12]:
# Define utility functions for multimedia processing

def extract_audio_from_video(video_path, output_audio_path):
    """Extract audio from video file using FFmpeg"""
    try:
        # Convert video_path and output_audio_path to strings in case they are Path objects
        video_path = str(video_path)
        output_audio_path = str(output_audio_path)
        
        # Use ffmpeg-python to extract audio
        (
            ffmpeg
            .input(video_path)
            .output(output_audio_path, acodec='pcm_s16le', ac=1, ar='16k')
            .run(quiet=True, overwrite_output=True)
        )
        print(f"Audio extracted successfully: {output_audio_path}")
        return True
    except Exception as e:
        print(f"Error extracting audio: {e}")
        return False

def segment_long_audio(audio_path, segment_length_ms=30000, output_dir=None):
    """Segment long audio files into smaller chunks"""
    try:
        # Load audio file
        audio = AudioSegment.from_file(audio_path)
        
        # Create output directory if not provided
        if output_dir is None:
            output_dir = os.path.join(os.path.dirname(audio_path), "segments")
        os.makedirs(output_dir, exist_ok=True)
        
        # Get base filename without extension
        base_filename = os.path.basename(audio_path).split('.')[0]
        
        # Calculate number of segments
        audio_length_ms = len(audio)
        num_segments = audio_length_ms // segment_length_ms + (1 if audio_length_ms % segment_length_ms > 0 else 0)
        
        segments = []
        
        # Split audio into segments
        for i in range(num_segments):
            start_ms = i * segment_length_ms
            end_ms = min((i + 1) * segment_length_ms, audio_length_ms)
            
            segment = audio[start_ms:end_ms]
            segment_path = os.path.join(output_dir, f"{base_filename}_segment_{i:03d}.wav")
            
            segment.export(segment_path, format="wav")
            segments.append({
                "path": segment_path,
                "start_ms": start_ms,
                "end_ms": end_ms,
                "duration_ms": end_ms - start_ms
            })
        
        print(f"Audio segmented into {len(segments)} parts")
        return segments
    except Exception as e:
        print(f"Error segmenting audio: {e}")
        return []

# Function to test the audio extraction and segmentation
def test_audio_processing(video_path=None):
    """Test audio processing functions with a sample video"""
    if video_path is None:
        print("Please provide a video path to test audio processing")
        return
    
    # Extract audio
    audio_path = os.path.join(AUDIO_DIR, os.path.basename(video_path).replace('.mp4', '.wav'))
    success = extract_audio_from_video(video_path, audio_path)
    
    if success:
        # Segment audio
        segments = segment_long_audio(audio_path, segment_length_ms=30000, output_dir=AUDIO_DIR / "segments")
        print(f"Created {len(segments)} audio segments")
        return segments
    else:
        print("Audio extraction failed, cannot proceed with segmentation")
        return []

# To test: test_audio_processing("path/to/video.mp4")

In [ ]:
def batch_extract_audio_from_videos():
    """Extract audio from all videos in the VIDEO_DIR directory"""
    # Get all video files
    video_files = list(VIDEO_DIR.glob("*.mp4")) + list(VIDEO_DIR.glob("*.mkv")) + list(VIDEO_DIR.glob("*.avi"))
    total_videos = len(video_files)
    
    if total_videos == 0:
        print("No video files found in", VIDEO_DIR)
        return []
    
    print(f"Found {total_videos} videos. Starting audio extraction...")
    
    # Create processed results list to track progress
    processed_files = []
    success_count = 0
    
    # Process each video
    for i, video_path in enumerate(video_files):
        video_name = video_path.stem  # Get filename without extension
        print(f"[{i+1}/{total_videos}] Processing {video_name}...")
        
        # Define output audio path
        audio_path = AUDIO_DIR / f"{video_name}.wav"
        
        # Extract audio
        try:
            success = extract_audio_from_video(video_path, audio_path)
            
            if success:
                success_count += 1
                processed_files.append({
                    "video_path": str(video_path),
                    "audio_path": str(audio_path),
                    "status": "success"
                })
                print(f"  ✓ Successfully extracted audio to {audio_path.name}")
            else:
                processed_files.append({
                    "video_path": str(video_path),
                    "status": "failed"
                })
                print(f"  ✗ Failed to extract audio from {video_path.name}")
        except Exception as e:
            print(f"  ✗ Error processing {video_path.name}: {e}")
            processed_files.append({
                "video_path": str(video_path),
                "status": "error",
                "error": str(e)
            })
    
    print(f"\nAudio extraction complete! Successfully processed {success_count}/{total_videos} videos.")
    
    # Save processing results to a log file
    log_path = PROCESSED_DIR / "audio_extraction_log.json"
    with open(log_path, "w") as f:
        json.dump({
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
            "total_videos": total_videos,
            "success_count": success_count,
            "processed_files": processed_files
        }, f, indent=2)
    
    print(f"Log saved to {log_path}")
    return processed_files

# Run the batch extraction process
# extracted_audio_files = batch_extract_audio_from_videos()

Found 15 videos. Starting audio extraction...
[1/15] Processing Multimodal_19853_shylaja.sharath_31_20250401112700078_Video_ENC...
Audio extracted successfully: data/processed/audio/Multimodal_19853_shylaja.sharath_31_20250401112700078_Video_ENC.wav
  ✓ Successfully extracted audio to Multimodal_19853_shylaja.sharath_31_20250401112700078_Video_ENC.wav
[2/15] Processing stable_diffusion_19853_shylaja.sharath_31_20250327084200249_Video_ENC...
Audio extracted successfully: data/processed/audio/stable_diffusion_19853_shylaja.sharath_31_20250327084200249_Video_ENC.wav
  ✓ Successfully extracted audio to stable_diffusion_19853_shylaja.sharath_31_20250327084200249_Video_ENC.wav
[3/15] Processing Lora&Qlora_19853_shylaja.sharath_31_20250318121200085_Video_ENC (1)...
Audio extracted successfully: data/processed/audio/Lora&Qlora_19853_shylaja.sharath_31_20250318121200085_Video_ENC (1).wav
  ✓ Successfully extracted audio to Lora&Qlora_19853_shylaja.sharath_31_20250318121200085_Video_ENC (1).wav


In [ ]:
def batch_segment_audio_files(segment_length_seconds=30):
    """Segment all extracted audio files into smaller chunks"""
    # Get all audio files in the AUDIO_DIR
    audio_files = list(AUDIO_DIR.glob("*.wav"))
    total_files = len(audio_files)
    
    if total_files == 0:
        print("No audio files found in", AUDIO_DIR)
        return []
    
    print(f"Found {total_files} audio files. Starting segmentation...")
    
    # Convert seconds to milliseconds
    segment_length_ms = segment_length_seconds * 1000
    
    # Create segments directory if it doesn't exist
    segments_dir = AUDIO_DIR / "segments"
    segments_dir.mkdir(exist_ok=True, parents=True)
    
    # Track processing results
    processed_files = []
    total_segments = 0
    
    # Process each audio file
    for i, audio_path in enumerate(audio_files):
        audio_name = audio_path.stem
        print(f"[{i+1}/{total_files}] Segmenting {audio_name}...")
        
        try:
            # Create a dedicated directory for this file's segments
            file_segments_dir = segments_dir / audio_name
            file_segments_dir.mkdir(exist_ok=True)
            
            # Segment the audio
            segments = segment_long_audio(
                audio_path,
                segment_length_ms=segment_length_ms,
                output_dir=file_segments_dir
            )
            
            # Track results
            total_segments += len(segments)
            processed_files.append({
                "audio_path": str(audio_path),
                "segments_dir": str(file_segments_dir),
                "segment_count": len(segments),
                "segments": segments,
                "status": "success"
            })
            
            print(f"  ✓ Created {len(segments)} segments in {file_segments_dir.name}/")
            
        except Exception as e:
            print(f"  ✗ Error segmenting {audio_path.name}: {e}")
            processed_files.append({
                "audio_path": str(audio_path),
                "status": "error",
                "error": str(e)
            })
    
    print(f"\nAudio segmentation complete! Created {total_segments} segments from {total_files} audio files.")
    
    # Save processing results to a log file
    log_path = PROCESSED_DIR / "audio_segmentation_log.json"
    with open(log_path, "w") as f:
        json.dump({
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
            "total_files": total_files,
            "total_segments": total_segments,
            "segment_length_seconds": segment_length_seconds,
            "processed_files": processed_files
        }, f, indent=2)
    
    print(f"Log saved to {log_path}")
    return processed_files

# Run the batch segmentation process
# segment_results = batch_segment_audio_files(segment_length_seconds=30)

Found 15 audio files. Starting segmentation...
[1/15] Segmenting TBT_6a_2020-09-22 09-49-32_TBTCon...
Audio segmented into 51 parts
  ✓ Created 51 segments in TBT_6a_2020-09-22 09-49-32_TBTCon/
[2/15] Segmenting Lora&Qlora_19853_shylaja.sharath_31_20250318125700082_Video_ENC...
Audio segmented into 111 parts
  ✓ Created 111 segments in Lora&Qlora_19853_shylaja.sharath_31_20250318125700082_Video_ENC/
[3/15] Segmenting binary_tree_traversal_5a_2020-09-15 09-04-51_BinTraversal...
Audio segmented into 70 parts
  ✓ Created 70 segments in binary_tree_traversal_5a_2020-09-15 09-04-51_BinTraversal/
[4/15] Segmenting Heap_8b_2020-09-26 09-34-27_HeapCode 00_00_09-00_56_50...
Audio segmented into 114 parts
  ✓ Created 114 segments in Heap_8b_2020-09-26 09-34-27_HeapCode 00_00_09-00_56_50/
[5/15] Segmenting expn_tree_7b_2020-09-25 12-12-37_ExprTReeCode 00_00_09-00_26_52...
Audio segmented into 54 parts
  ✓ Created 54 segments in expn_tree_7b_2020-09-25 12-12-37_ExprTReeCode 00_00_09-00_26_52/
[6/1

In [ ]:
def process_all_videos(extract_audio=True, segment_audio=True, segment_length_seconds=30):
    """Process all videos in the dataset by extracting audio and optionally segmenting it"""
    results = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "extract_audio": None,
        "segment_audio": None
    }
    
    # Extract audio from all videos
    if extract_audio:
        print("STEP 1: Extracting audio from all videos")
        print("-" * 50)
        extracted_files = batch_extract_audio_from_videos()
        results["extract_audio"] = {
            "success_count": sum(1 for f in extracted_files if f["status"] == "success"),
            "total_videos": len(extracted_files)
        }
        print("\n")
    
    # Segment all audio files
    if segment_audio:
        print("STEP 2: Segmenting all audio files")
        print("-" * 50)
        segment_results = batch_segment_audio_files(segment_length_seconds)
        results["segment_audio"] = {
            "success_count": sum(1 for f in segment_results if f["status"] == "success"),
            "total_files": len(segment_results),
            "total_segments": sum(f["segment_count"] for f in segment_results if "segment_count" in f)
        }
    
    print("\nProcessing Summary:")
    print("-" * 50)
    if extract_audio:
        print(f"Audio extraction: {results['extract_audio']['success_count']}/{results['extract_audio']['total_videos']} videos")
    if segment_audio:
        print(f"Audio segmentation: {results['segment_audio']['total_segments']} segments from {results['segment_audio']['success_count']}/{results['segment_audio']['total_files']} files")
    
    return results

# Process all videos in the dataset
processing_results = process_all_videos(extract_audio=True, segment_audio=True, segment_length_seconds=30)

## 4. AI Model Configuration

In this section, we configure the necessary AI models for speech-to-text transcription and language processing.

In [ ]:
# Set up API credentials
# Note: Using only Hugging Face models for this implementation

# Hugging Face credentials
def setup_huggingface():
    print("Setting up Hugging Face access...")
    
    # Use the provided Hugging Face token
    hf_token = os.environ.get("HUGGINGFACE_TOKEN", "keeep ur API Key")
    
    if not hf_token:
        print("Warning: HUGGINGFACE_TOKEN not found. Some models may not be accessible.")
        return False
    
    try:
        login(token=hf_token)
        print("Hugging Face login successful!")
        return True
    except Exception as e:
        print(f"Failed to log in to Hugging Face: {e}")
        return False

# Function to load and configure a speech transcription model from Hugging Face
def load_whisper_model(model_id="openai/whisper-small"):
    """Load a Whisper model for speech recognition"""
    try:
        print(f"Loading speech-to-text model: {model_id}")
        
        # Load processor
        processor = AutoProcessor.from_pretrained(model_id)
        
        # Load model
        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
        # Create a pipeline - don't specify device when using accelerate with device_map="auto"
        pipe = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor
            # Remove the device argument as it conflicts with accelerate
        )
        
        print(f"Speech-to-text model loaded: {model_id}")
        return pipe
    except Exception as e:
        print(f"Error loading whisper model: {e}")
        return None

# Function to load a language model for summarization and note generation
def load_language_model(model_id="google/flan-t5-base"):
    """Load a language model for text generation and summarization"""
    try:
        print(f"Loading language model: {model_id}")
        
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto"
        )
        
        print(f"Language model loaded: {model_id}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading language model: {e}")
        return None, None

# Also can use text generation pipeline as an alternative approach
def load_text_generation_model(model_id="google/flan-t5-large"):
    """Load a text generation pipeline from Hugging Face"""
    try:
        print(f"Loading text generation model: {model_id}")
        
        # Set device if not provided
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Create a text generation pipeline
        text_generator = pipeline(
            "text2text-generation",
            model=model_id,
            device=device
        )
        
        print(f"Text generation model loaded: {model_id}")
        return text_generator
    except Exception as e:
        print(f"Error loading text generation model: {e}")
        return None

# Set up Hugging Face access - uncomment to use
hf_setup = setup_huggingface()

Setting up Hugging Face access...
Hugging Face login successful!
Hugging Face login successful!


## 5. Video Transcription Pipeline

Implement the complete pipeline for extracting audio from videos and converting speech to text.

In [24]:
# Complete transcription pipeline
def transcribe_video(video_path, transcription_model=None, max_segment_length_sec=30):
    """
    Process a video file to generate a transcript with timestamps
    
    Args:
        video_path: Path to the video file
        transcription_model: Pre-loaded transcription model pipeline
        max_segment_length_sec: Maximum segment length in seconds
    
    Returns:
        Dictionary containing the transcript and metadata
    """
    video_filename = os.path.basename(video_path)
    video_name = os.path.splitext(video_filename)[0]
    
    print(f"Processing video: {video_filename}")
    
    # 1. Extract audio from video
    audio_path = os.path.join(AUDIO_DIR, f"{video_name}.wav")
    extraction_success = extract_audio_from_video(video_path, audio_path)
    
    if not extraction_success:
        return {"error": "Failed to extract audio from video"}
    
    # 2. Segment the audio for better transcription
    segment_dir = os.path.join(AUDIO_DIR, f"{video_name}_segments")
    os.makedirs(segment_dir, exist_ok=True)
    
    segments = segment_long_audio(
        audio_path, 
        segment_length_ms=max_segment_length_sec * 1000, 
        output_dir=segment_dir
    )
    
    if not segments:
        return {"error": "Failed to segment audio"}
    
    # 3. Load transcription model if not provided
    if transcription_model is None:
        print("Loading transcription model...")
        transcription_model = load_whisper_model()
        
    if transcription_model is None:
        return {"error": "Failed to load transcription model"}
    
    # 4. Transcribe each segment
    print(f"Transcribing {len(segments)} audio segments...")
    
    full_transcript = []
    
    for i, segment in enumerate(segments):
        print(f"Transcribing segment {i+1}/{len(segments)}...")
        
        try:
            # Perform transcription
            result = transcription_model(
                segment["path"],
                return_timestamps=True,
                generate_kwargs={"language": "en"}
            )
            
            # Calculate absolute timestamps
            segment_start_sec = segment["start_ms"] / 1000
            
            # Process chunk results
            if "chunks" in result:
                for chunk in result["chunks"]:
                    # Adjust timestamps to be relative to the full video
                    start_time = segment_start_sec + chunk["timestamp"][0] if chunk["timestamp"][0] is not None else segment_start_sec
                    end_time = segment_start_sec + chunk["timestamp"][1] if chunk["timestamp"][1] is not None else None
                    
                    full_transcript.append({
                        "text": chunk["text"],
                        "start_time": start_time,
                        "end_time": end_time
                    })
            else:
                # If no chunks, use the whole segment
                full_transcript.append({
                    "text": result["text"],
                    "start_time": segment_start_sec,
                    "end_time": segment_start_sec + (segment["duration_ms"] / 1000)
                })
                
        except Exception as e:
            print(f"Error transcribing segment {i+1}: {e}")
            # Continue with other segments even if one fails
    
    # 5. Combine and save the transcript
    transcript_path = os.path.join(TRANSCRIPTS_DIR, f"{video_name}_transcript.json")
    
    transcript_result = {
        "video_name": video_name,
        "video_path": str(video_path),
        "duration_seconds": segments[-1]["end_ms"] / 1000 if segments else 0,
        "transcript": full_transcript,
        "transcript_text": " ".join([item["text"] for item in full_transcript]),
    }
    
    with open(transcript_path, 'w') as f:
        json.dump(transcript_result, f, indent=2)
    
    print(f"Transcript saved to: {transcript_path}")
    return transcript_result

## 5.1 Batch Transcription Process

After successfully extracting and segmenting audio from all videos, and processing the slides, we'll now transcribe all the audio files to generate text transcripts.

In [25]:
import re  # Import for the evaluate_lecture_notes function

def batch_transcribe_audio_files(use_segments=False, whisper_model=None):
    """
    Batch process all audio files to generate transcripts
    
    Args:
        use_segments: Whether to use segmented audio files (True) or full audio files (False)
        whisper_model: Pre-loaded Whisper model, will load if None
    
    Returns:
        Dictionary of transcription results
    """
    print("Starting batch transcription process...")
    
    # 1. Load the transcription model if not provided
    if whisper_model is None:
        print("Loading Whisper model...")
        whisper_model = load_whisper_model()
        if whisper_model is None:
            return {"error": "Failed to load Whisper model"}
    
    # 2. Get the list of files to transcribe
    audio_files = []
    if not use_segments:
        # Use full audio files (non-segmented) for transcript generation
        audio_files = list(AUDIO_DIR.glob("*.wav"))
    else:
        # Use segmented files - more complex but can be more efficient for long files
        # Implementation would depend on your specific needs
        pass
    
    total_files = len(audio_files)
    if total_files == 0:
        print("No audio files found to transcribe")
        return {"error": "No audio files found"}
    
    print(f"Found {total_files} audio files to transcribe")
    
    # 3. Process each file
    results = []
    success_count = 0
    
    for i, audio_path in enumerate(audio_files):
        print(f"[{i+1}/{total_files}] Transcribing {audio_path.stem}...")
        
        # Derive corresponding video path for reference
        video_name = audio_path.stem
        possible_video_extensions = [".mp4", ".mkv", ".avi"]
        video_path = None
        
        for ext in possible_video_extensions:
            potential_path = VIDEO_DIR / f"{video_name}{ext}"
            if potential_path.exists():
                video_path = potential_path
                break
        
        if video_path is None:
            # If exact match not found, try to find a video with this name as part of its filename
            for ext in possible_video_extensions:
                matching_files = list(VIDEO_DIR.glob(f"*{video_name}*{ext}"))
                if matching_files:
                    video_path = matching_files[0]
                    break
        
        # Fall back to using the audio file path as reference
        if video_path is None:
            video_path = audio_path
        
        try:
            # Skip the audio extraction and segmentation steps since we already have the audio
            # We'll modify the transcribe_video function to work directly with audio files
            
            # Create the segments directory if it doesn't exist yet
            segment_dir = os.path.join(AUDIO_DIR, f"{video_name}_segments")
            os.makedirs(segment_dir, exist_ok=True)
            
            # Check if segments already exist
            segments = list(Path(segment_dir).glob("*.wav"))
            if not segments:
                # Create segments if they don't exist
                segments = segment_long_audio(
                    audio_path,
                    segment_length_ms=30000,
                    output_dir=segment_dir
                )
                segments = [{'path': s["path"], 'start_ms': s["start_ms"], 'end_ms': s["end_ms"], 'duration_ms': s["duration_ms"]} for s in segments]
            else:
                # Convert existing segment paths to the expected format
                sorted_segments = sorted(segments, key=lambda x: int(str(x).split('_')[-1].split('.')[0]))
                segments = []
                total_duration = 0
                segment_length_ms = 30000  # Default length in ms
                
                for i, segment_path in enumerate(sorted_segments):
                    start_ms = i * segment_length_ms
                    # For the last segment, approximate the duration
                    try:
                        audio_segment = AudioSegment.from_file(segment_path)
                        duration_ms = len(audio_segment)
                    except:
                        duration_ms = segment_length_ms
                        
                    end_ms = start_ms + duration_ms
                    total_duration += duration_ms
                    
                    segments.append({
                        'path': str(segment_path),
                        'start_ms': start_ms,
                        'end_ms': end_ms,
                        'duration_ms': duration_ms
                    })
            
            # Process segments for transcription
            full_transcript = []
            
            for i, segment in enumerate(segments):
                print(f"  Transcribing segment {i+1}/{len(segments)}...")
                
                try:
                    # Perform transcription
                    result = whisper_model(
                        segment["path"],
                        return_timestamps=True,
                        generate_kwargs={"language": "en"}
                    )
                    
                    # Calculate absolute timestamps
                    segment_start_sec = segment["start_ms"] / 1000 if isinstance(segment["start_ms"], (int, float)) else 0
                    
                    # Process chunk results
                    if "chunks" in result:
                        for chunk in result["chunks"]:
                            # Adjust timestamps to be relative to the full video
                            start_time = segment_start_sec + chunk["timestamp"][0] if chunk["timestamp"][0] is not None else segment_start_sec
                            end_time = segment_start_sec + chunk["timestamp"][1] if chunk["timestamp"][1] is not None else None
                            
                            full_transcript.append({
                                "text": chunk["text"],
                                "start_time": start_time,
                                "end_time": end_time
                            })
                    else:
                        # If no chunks, use the whole segment
                        full_transcript.append({
                            "text": result["text"],
                            "start_time": segment_start_sec,
                            "end_time": segment_start_sec + (segment["duration_ms"] / 1000 if isinstance(segment["duration_ms"], (int, float)) else 30)
                        })
                        
                except Exception as e:
                    print(f"    ✗ Error transcribing segment {i+1}: {e}")
                    # Continue with other segments even if one fails
            
            # Save the transcript
            os.makedirs(TRANSCRIPTS_DIR, exist_ok=True)
            transcript_path = os.path.join(TRANSCRIPTS_DIR, f"{video_name}_transcript.json")
            
            transcript_result = {
                "video_name": video_name,
                "video_path": str(video_path),
                "duration_seconds": (segments[-1]["end_ms"] / 1000) if segments and "end_ms" in segments[-1] else 0,
                "transcript": full_transcript,
                "transcript_text": " ".join([item["text"] for item in full_transcript]),
            }
            
            with open(transcript_path, 'w') as f:
                json.dump(transcript_result, f, indent=2)
            
            success_count += 1
            results.append({
                "audio": str(audio_path),
                "transcript": str(transcript_path),
                "word_count": len(transcript_result["transcript_text"].split()),
                "status": "success"
            })
            print(f"  ✓ Successfully transcribed {audio_path.name} ({len(full_transcript)} segments)")
                
        except Exception as e:
            print(f"  ✗ Error transcribing {audio_path.name}: {e}")
            results.append({
                "audio": str(audio_path),
                "status": "error",
                "error": str(e)
            })
    
    print(f"\nTranscription complete! Successfully processed {success_count}/{total_files} files.")
    
    # Save transcription log
    log_path = PROCESSED_DIR / "transcription_log.json"
    log_data = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "total_files": total_files,
        "success_count": success_count,
        "results": results
    }
    
    with open(log_path, "w") as f:
        json.dump(log_data, f, indent=2)
    
    print(f"Log saved to {log_path}")
    return log_data

In [ ]:
# Run the audio transcription process
print("Loading Whisper model for transcription...")
whisper_model = load_whisper_model(model_id="openai/whisper-small")

if whisper_model is not None:
    print("Starting batch transcription...")
    transcription_results = batch_transcribe_audio_files(whisper_model=whisper_model)
else:
    print("Failed to load Whisper model. Please check your environment configuration.")

Loading Whisper model for transcription...
Loading speech-to-text model: openai/whisper-small


Device set to use mps


Speech-to-text model loaded: openai/whisper-small
Starting batch transcription...
Starting batch transcription process...
Found 15 audio files to transcribe
[1/15] Transcribing TBT_6a_2020-09-22 09-49-32_TBTCon...
Audio segmented into 51 parts
  Transcribing segment 1/51...


/Users/koushikmithul/anaconda3/envs/LLM/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=en, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=en.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
/Users/koushikmithul/anaconda3/envs/LLM/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input

  Transcribing segment 2/51...


...

  Transcribing segment 54/71...


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
/Users/koushikmithul/anaconda3/envs/LLM/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/Users/koushikmithul/anaconda3/envs/LLM/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


  Transcribing segment 55/71...


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
/Users/koushikmithul/anaconda3/envs/LLM/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/Users/koushikmithul/anaconda3/envs/LLM/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


  Transcribing segment 56/71...


## 6. PowerPoint Processing Pipeline

Extract and process content from PowerPoint slides to supplement the transcribed lectures.

In [3]:
def extract_slide_content(slide_path):
    """
    Extract content from a PowerPoint presentation
    
    Args:
        slide_path: Path to the PowerPoint file
    
    Returns:
        Dictionary containing structured slide content
    """
    try:
        presentation = Presentation(slide_path)
        slide_content = []
        
        # Extract presentation metadata
        metadata = {
            "title": os.path.splitext(os.path.basename(slide_path))[0],
            "slide_count": len(presentation.slides)
        }
        
        # Process each slide
        for idx, slide in enumerate(presentation.slides):
            slide_data = {
                "slide_number": idx + 1,
                "title": "",
                "content": [],
                "notes": "",
                "has_images": False,
                "image_count": 0
            }
            
            # Extract slide title (if available)
            if slide.shapes.title:
                slide_data["title"] = slide.shapes.title.text
            
            # Extract text content from shapes
            for shape in slide.shapes:
                if hasattr(shape, "text") and shape.text.strip():
                    # Skip if this is the title we already captured
                    if shape == slide.shapes.title:
                        continue
                    
                    slide_data["content"].append({
                        "type": "text",
                        "text": shape.text.strip()
                    })
                
                # Check if the shape contains an image
                if shape.shape_type == 13:  # MSO_SHAPE_TYPE.PICTURE
                    slide_data["has_images"] = True
                    slide_data["image_count"] += 1
            
            # Extract notes if available
            if slide.notes_slide and slide.notes_slide.notes_text_frame.text.strip():
                slide_data["notes"] = slide.notes_slide.notes_text_frame.text.strip()
            
            slide_content.append(slide_data)
        
        # Save the extracted content
        output_filename = os.path.splitext(os.path.basename(slide_path))[0]
        output_path = os.path.join(PROCESSED_DIR, f"{output_filename}_content.json")
        
        result = {
            "metadata": metadata,
            "slides": slide_content
        }
        
        with open(output_path, 'w') as f:
            json.dump(result, f, indent=2)
            
        print(f"Extracted content from {len(slide_content)} slides")
        print(f"Saved slide content to: {output_path}")
        
        return result
        
    except Exception as e:
        print(f"Error processing PowerPoint file: {e}")
        return None

# Function to extract text content from slides for use in note generation
def get_slide_text_content(slide_data):
    """Extract plain text content from slide data"""
    if not slide_data:
        return ""
    
    content_parts = []
    
    # Add metadata
    content_parts.append(f"Presentation: {slide_data['metadata']['title']}")
    content_parts.append(f"Total Slides: {slide_data['metadata']['slide_count']}")
    content_parts.append("")
    
    # Process each slide
    for slide in slide_data["slides"]:
        # Add slide header
        content_parts.append(f"Slide {slide['slide_number']}: {slide['title']}")
        
        # Add content
        for item in slide["content"]:
            if item["type"] == "text":
                content_parts.append(item["text"])
        
        # Add notes
        if slide["notes"]:
            content_parts.append(f"Notes: {slide['notes']}")
        
        # Add separator between slides
        content_parts.append("-" * 40)
    
    return "\n".join(content_parts)

## 6.1 Batch Slide Processing

Let's process all the slide files to extract their content for the lecture notes.

In [6]:
def batch_process_slides():
    """Process all PowerPoint and PDF slides in the slides directory"""
    # Find all slide files
    pptx_files = list(SLIDES_DIR.glob("*.pptx"))
    ppt_files = list(SLIDES_DIR.glob("*.ppt"))
    pdf_files = list(SLIDES_DIR.glob("*.pdf"))
    
    all_slides = pptx_files + ppt_files + pdf_files
    total_slides = len(all_slides)
    
    if total_slides == 0:
        print("No slide files found in", SLIDES_DIR)
        return []
    
    print(f"Found {total_slides} slide files. Starting content extraction...")
    
    # Process slides
    results = []
    success_count = 0
    
    for i, slide_path in enumerate(all_slides):
        print(f"[{i+1}/{total_slides}] Processing {slide_path.stem}...")
        
        try:
            if slide_path.suffix.lower() in [".pptx", ".ppt"]:
                # Process PowerPoint file
                result = extract_slide_content(slide_path)
                
                if result is not None:
                    success_count += 1
                    results.append({
                        "slide_path": str(slide_path),
                        "content_file": f"{slide_path.stem}_content.json",
                        "slide_count": result["metadata"]["slide_count"],
                        "status": "success"
                    })
                    print(f"  ✓ Successfully extracted content from {slide_path.name}")
                else:
                    results.append({
                        "slide_path": str(slide_path),
                        "status": "failed"
                    })
                    print(f"  ✗ Failed to extract content from {slide_path.name}")
            
            elif slide_path.suffix.lower() == ".pdf":
                # Process PDF file (extract text content)
                output_path = PROCESSED_DIR / f"{slide_path.stem}_content.json"
                
                try:
                    # Open the PDF
                    pdf_document = fitz.open(slide_path)
                    page_count = len(pdf_document)
                    
                    # Extract content from each page
                    pdf_content = []
                    for page_num, page in enumerate(pdf_document):
                        text = page.get_text()
                        pdf_content.append({
                            "slide_number": page_num + 1,
                            "title": f"Page {page_num + 1}",  # PDF doesn't have explicit titles
                            "content": [{"type": "text", "text": text}],
                            "notes": "",
                            "has_images": len(page.get_images()) > 0,
                            "image_count": len(page.get_images())
                        })
                    
                    result = {
                        "metadata": {
                            "title": slide_path.stem,
                            "slide_count": page_count
                        },
                        "slides": pdf_content
                    }
                    
                    # Save extracted content
                    with open(output_path, 'w') as f:
                        json.dump(result, f, indent=2)
                    
                    success_count += 1
                    results.append({
                        "slide_path": str(slide_path),
                        "content_file": output_path.name,
                        "slide_count": page_count,
                        "status": "success"
                    })
                    print(f"  ✓ Successfully extracted content from PDF {slide_path.name}")
                    
                except Exception as e:
                    print(f"  ✗ Error processing PDF {slide_path.name}: {e}")
                    results.append({
                        "slide_path": str(slide_path),
                        "status": "error",
                        "error": str(e)
                    })
            else:
                print(f"  ! Unsupported file format: {slide_path.suffix}")
                results.append({
                    "slide_path": str(slide_path),
                    "status": "skipped",
                    "reason": "Unsupported format"
                })
                
        except Exception as e:
            print(f"  ✗ Error processing {slide_path.name}: {e}")
            results.append({
                "slide_path": str(slide_path),
                "status": "error",
                "error": str(e)
            })
    
    print(f"\nSlide processing complete! Successfully processed {success_count}/{total_slides} files.")
    
    # Save processing log
    log_path = PROCESSED_DIR / "slides_processing_log.json"
    log_data = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "total_slides": total_slides,
        "success_count": success_count,
        "results": results
    }
    
    with open(log_path, "w") as f:
        json.dump(log_data, f, indent=2)
        
    print(f"Log saved to {log_path}")
    return results

# Run the batch slide processing
slide_processing_results = batch_process_slides()

Found 14 slide files. Starting content extraction...
[1/14] Processing Tree_traversal_class9_Unit3_Trees_naryTraversal...
Extracted content from 13 slides
Saved slide content to: data/processed/Tree_traversal_class9_Unit3_Trees_naryTraversal_content.json
  ✓ Successfully extracted content from Tree_traversal_class9_Unit3_Trees_naryTraversal.pptx
[2/14] Processing BST_Class3_Unit3_Trees_BSTDeletion...
Extracted content from 10 slides
Saved slide content to: data/processed/BST_Class3_Unit3_Trees_BSTDeletion_content.json
  ✓ Successfully extracted content from BST_Class3_Unit3_Trees_BSTDeletion.pptx
[3/14] Processing BST_Class2_Unit3_Tree_BST_DynamicInsert...
Extracted content from 8 slides
Saved slide content to: data/processed/BST_Class2_Unit3_Tree_BST_DynamicInsert_content.json
  ✓ Successfully extracted content from BST_Class2_Unit3_Tree_BST_DynamicInsert.pptx
[4/14] Processing BST_Class4_Unit3_Trees_BST_ArrayInsert...
Extracted content from 7 slides
Saved slide content to: data/proce

## 8. Evaluation of Generated Lecture Notes

Implement evaluation metrics to assess the quality and usefulness of the generated notes.

In [ ]:
def evaluate_lecture_notes(generated_notes, reference_transcript=None, reference_slides=None):
    """
    Evaluate the quality of generated lecture notes
    
    Args:
        generated_notes: The generated lecture notes text
        reference_transcript: Original transcript for comparison
        reference_slides: Original slide content for comparison
    
    Returns:
        Dictionary containing evaluation metrics
    """
    evaluation = {
        "quantitative_metrics": {},
        "qualitative_assessment": {},
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Quantitative metrics
    if generated_notes:
        # Basic metrics
        word_count = len(generated_notes.split())
        sentence_count = len([s for s in generated_notes.split('.') if s.strip()])
        avg_sentence_length = word_count / max(sentence_count, 1)
        
        # Readability metrics (simplified)
        complex_words = len([w for w in generated_notes.split() if len(w) > 6])
        complexity_percentage = (complex_words / max(word_count, 1)) * 100
        
        evaluation["quantitative_metrics"] = {
            "word_count": word_count,
            "sentence_count": sentence_count,
            "average_sentence_length": avg_sentence_length,
            "complex_word_percentage": complexity_percentage
        }
        
        # Structure assessment
        has_title = "# " in generated_notes or "Title:" in generated_notes
        has_sections = bool(re.findall(r'^\d+\.\s', generated_notes, re.MULTILINE))
        has_summary = "Summary" in generated_notes or "Conclusion" in generated_notes
        
        evaluation["qualitative_assessment"] = {
            "has_title": has_title,
            "has_structured_sections": has_sections,
            "has_summary": has_summary
        }
        
        # Content coverage (if reference material provided)
        if reference_transcript:
            # Simple keyword overlap analysis
            transcript_keywords = set([w.lower() for w in re.findall(r'\b\w{4,}\b', reference_transcript)])
            notes_keywords = set([w.lower() for w in re.findall(r'\b\w{4,}\b', generated_notes)])
            
            keyword_overlap = len(transcript_keywords.intersection(notes_keywords))
            coverage_ratio = keyword_overlap / max(len(transcript_keywords), 1)
            
            evaluation["content_coverage"] = {
                "keyword_overlap_count": keyword_overlap,
                "keyword_coverage_ratio": coverage_ratio
            }
    
    return evaluation